<a href="https://bit.ly/Sports2D_Colab" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sports2D

When seeking to optimize a sports movement, it is often useful to compute joint positions, or joint and segment angles. \
**[Sports2D](https://github.com/davidpagnon/Sports2D)** offers a way to compute them from a video, on any platform, including your smartphone.\
Run cells one after another by clicking on the ▶️ button.

</br>

<img src="https://raw.githubusercontent.com/davidpagnon/Sports2D/main/Content/demo_gif.gif" title='Demonstration of Sports2D with OpenPose.'  width="760">


## Additional information

Please find more information about how it works under the hood [here](https://github.com/davidpagnon/Sports2D).

While you're there, do not hesitate to hit the ⭐Star button, I would appreciate the support! \
If you use Sports2D, please cite:

     @misc{Pagnon2023,
       author = {Pagnon, David},
       title = {Sports2D - Angles from video},
       year = {2023},
       doi= {10.5281/zenodo.7903963},
       publisher = {GitHub},
       journal = {GitHub repository},
       howpublished = {\url{https://github.com/davidpagnon/Sports2D}},
     }

</br>

**Warning:**
- Results are only as good as the pose estimation algorithm, i.e., they are not perfect. Moreover, they will not be good if your video is blurry. Finally, they are acceptable only if the persons move in the 2D plane, from right to left or from left to right. If you need research-grade 3D markerless kinematics, consider using several cameras with [Pose2Sim](https://github.com/perfanalytics/pose2sim) instead.
- Your data will be sent to the Google servers, which do not follow the European GDPR requirements regarding privacy. If you don't want this, you can [run Sports 2D on your computer](https://github.com/davidpagnon/Sports2D) instead.
- The server disconnects after 90 minutes of idle time, and after a few hours in any case. Make sure you have downloaded your results beforehand.
- Unless you subscribed, there is a time limit on the usage of GPUs on Colab. However, these are only mandatory if you run with onnxruntime-gpu
backend. If you run with cpu backend, you can safely ignore useage of GPUs on Colab.
- Currently, Colab does not allow real-time analytics using a webcam. You must use a local environment to analyze in real-time.

# Installation

In [ ]:
#@markdown You can choose to:
#@markdown - **Install just once**. Full install takes about 2 Go on your Google Drive,\
#@markdown but next times the environment should be set up in about 3 minutes, including on your smartphone.
#@markdown - **Install every time**. Full install takes about 5 minute, \
#@markdown but no connexion to Google Drive nor storage space is needed.
installation_type = 'Install once' #@param ["Install once", "Install every time"]

#@markdown Do you want to use GPU acceleration? (Recommended if available)
use_gpu = True #@param {type:"boolean"}

#@markdown <br>

import os
import sys
from google.colab import files, drive

def connect_to_Gdrive():
    """Connect to Google Drive and set up Sports2D directory."""
    print("Connecting to Google Drive...")
    drive.mount('/content/drive')
    if not os.path.exists(sports2d_path):
        !mkdir -p '/content/drive/My Drive/Sports2D'
    else:
        print(f"{sports2d_path} already exists.")
        !cp -r $sports2d_path '/content/drive/My Drive'
        !rm -r $sports2d_path

    # Remove existing symlink if present
    if os.path.islink(sports2d_path):
        os.unlink(sports2d_path)

    # Create new symlink
    os.symlink('/content/drive/My Drive/Sports2D', sports2d_path)
    sys.path.insert(0, sports2d_path)

def install_Sports2D_in_GDrive():
    """Install Sports2D in Google Drive."""
    print("Installing Sports2D in Google Drive...")
    if not os.path.exists(python_path):
        !PYTHONUSERBASE=$python_path pip install --user git+https://github.com/hunminkim98/Sports2D.git
    if use_gpu:
        !PYTHONUSERBASE=$python_path pip install --user onnxruntime-gpu
    else:
        print('Sports2D already installed.')
    # Add the GDrive python installation path to PYTHONPATH
    sys.path.append(os.path.join(python_path, "lib/python3.10/site-packages"))

def install_gpu_dependencies():
    """Install GPU dependencies."""
    print("Installing GPU dependencies...")
    !pip install onnxruntime-gpu # Only needed it for using GPU on Colab.

# Define paths
sports2d_path = '/content/Sports2D'
python_path = os.path.join(sports2d_path, 'Sports2D_python')
video_path = os.path.join(sports2d_path, 'Sports2D_videos')
result_path = os.path.join(sports2d_path, 'Sports2D_results')

print(f"video_path : {video_path}")
print(f"result_path : {result_path}")

# Use the right Python version
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
# GDrive tools
!apt-get -qq install xattr &> /dev/null

if installation_type == "Install every time":
    print('Installing Sports2D')
    !pip install git+https://github.com/hunminkim98/Sports2D.git
    if use_gpu:
        install_gpu_dependencies()
elif installation_type == "Install once":
    connect_to_Gdrive()
    install_Sports2D_in_GDrive()

print("Done.")

video_path : /content/Sports2D/Sports2D_videos
result_path : /content/Sports2D/Sports2D_results
update-alternatives: using /usr/bin/python3.10 to provide /usr/bin/python3 (python3) in auto mode
Connecting to Google Drive...
Mounted at /content/drive
Installing Sports2D in Google Drive...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.2 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Done.


# Upload videos

In [ ]:
from IPython.display import display, HTML
import toml

#@markdown Upload your videos if needed, and convert them if needed.\
#@markdown **Note**: As this takes time, you should probably try to trim your videos beforehand

def upload_videos():
    if not os.path.exists(video_path):
        !mkdir -p $video_path
    %cd $video_path
    uploaded = files.upload()
    return list(uploaded.keys())

def is_video_file(filename):
    video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.wmv', '.flv', '.webm']
    return any(filename.lower().endswith(ext) for ext in video_extensions)

def convert_vertical_iphone_vids(vids):
    # Pose algorithms do not see the rotation in certain videos taken in portrait mode. Convert them
    for i, vid in enumerate(vids):
        iphone_portrait = !ffprobe -loglevel error -select_streams v:0 -show_entries stream_tags=rotate -of default=nw=1:nk=1 -i $vid
        if iphone_portrait:
            vid_out = os.path.splitext(vid)[0]+'_converted'+os.path.splitext(vid)[1]
            print(f'{vid} is rotated. Converting it to {vid_out}...')
            !ffmpeg -i $vid $vid_out -loglevel quiet
            vids[i] = vid_out
            os.remove(vid)
    return vids

# Retrieve Sports2D Config file
if not os.path.exists(result_path):
    !mkdir -p $result_path

if installation_type == 'Install every time':
    !cp /usr/local/lib/python3.10/dist-packages/Sports2D/Demo/Config_demo.toml $result_path/Config.toml
elif installation_type == 'Install once':
    !cp $python_path/lib/python3.10/site-packages/Sports2D/Demo/Config_demo.toml $result_path/Config.toml
%cd $result_path
config_dict = toml.load('Config.toml')

# Upload videos and get the list of uploaded files
new_uploads = upload_videos()

# Process only newly uploaded videos
if new_uploads:
    vids = [f for f in new_uploads if is_video_file(f)]
    vids = convert_vertical_iphone_vids(vids)
else:
    vids = []

# If no new videos were uploaded or no valid video files were found, use demo.mp4
if not vids:
    if installation_type == 'Install every time':
        !cp /usr/local/lib/python3.10/dist-packages/Sports2D/Demo/demo.mp4 $video_path/demo.mp4
    elif installation_type == 'Install once':
        !cp $python_path/lib/python3.10/site-packages/Sports2D/Demo/demo.mp4 $video_path/demo.mp4
    vids = ['demo.mp4']

print(f'{vids} ready to be processed.')

# Display the data type
display(HTML("<b>Data type: video</b>"))

/content/drive/My Drive/Sports2D/Sports2D_results
/content/drive/My Drive/Sports2D/Sports2D_videos


['demo.mp4'] ready to be processed.


# Run Sports2D


## Settings (optional)

### Pose settings

In [ ]:
from enum import Flag

#@markdown ### Pose Settings
det_frequency = 1 #@param {type:"slider", min:1, max:30, step:1}
mode = "lightweight" #@param ["performance", "balanced", "lightweight"]
keypoints_threshold = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.1}

# Colab not supported cv2.show
display_detection = False
show_plots = False

#@markdown ### Video Settings
time_range = [] #@param {type:"raw"}

# Update pose settings
config_dict['pose_advanced'].update({
    'det_frequency': det_frequency,
    'mode': mode,
    'keypoints_threshold': keypoints_threshold,
})

config_dict['pose'].update({
    'time_range': time_range,
    'display_detection': display_detection,
})

config_dict['compute_angles_advanced'].update({
    'show_plots': show_plots
})
config_dict['pose_advanced'].update({
    'show_plots': show_plots
})

# Update project settings
config_dict['project'].update({
    'video_dir': video_path,
    'video_input': vids,
    'result_dir': result_path
})

# Save updated config
with open('Config.toml', 'w') as f:
    toml.dump(config_dict, f)

# Display selected options
display(HTML(f"""
<h3>Selected Configuration:</h3>
<ul>
    <li>Data Type: video</li>
    <li>Detection Frequency: {det_frequency}</li>
    <li>Mode: {mode}</li>
    <li>Keypoints Threshold: {keypoints_threshold}</li>
    <li>Result Directory: {result_path}</li>
    <li>Video Files: {', '.join(vids)}</li>
    <li>Time Range: {time_range}</li>
</ul>
"""))

# Check if video files exist
videos_full_path = [os.path.join(video_path, video_name) for video_name in vids]
for video_full_path in videos_full_path:
    if os.path.isfile(video_full_path):
        print(f'Video found at {video_full_path}')
    else:
        print(f'Video not found at {video_full_path}')

Video found at /content/Sports2D/Sports2D_videos/demo.mp4


### Angle settings

In [ ]:
#@markdown - Select **joint angles** among:
right_ankle = True #@param {type:"boolean"}
left_ankle = True #@param {type:"boolean"}
right_knee = True #@param {type:"boolean"}
left_knee = True #@param {type:"boolean"}
right_hip = True #@param {type:"boolean"}
left_hip = True #@param {type:"boolean"}
right_shoulder = True #@param {type:"boolean"}
left_shoulder = True #@param {type:"boolean"}
right_elbow = True #@param {type:"boolean"}
left_elbow = True #@param {type:"boolean"}

joint_angles_var = [right_ankle, left_ankle, right_knee, left_knee, right_hip, left_hip, right_shoulder, left_shoulder, right_elbow, left_elbow]
joint_angles_dict = {'Right ankle':right_ankle, 'Left ankle': left_ankle, 'Right knee':right_knee, 'Left knee':left_knee, 'Right hip':right_hip, 'Left hip':left_hip, 'Right shoulder':right_shoulder, 'Left shoulder':left_shoulder, 'Right elbow':right_elbow, 'Left elbow':left_elbow}
joint_angles_val = [key for key in joint_angles_dict.keys() if joint_angles_dict[key]]

config_dict.get('compute_angles').update({'joint_angles':joint_angles_val})

#@markdown <br>

#@markdown - Select **segment angles** among:
right_foot = True #@param {type:"boolean"}
left_foot = True #@param {type:"boolean"}
right_shank = True #@param {type:"boolean"}
left_shank = True #@param {type:"boolean"}
right_thigh = True #@param {type:"boolean"}
left_thigh = True #@param {type:"boolean"}
trunk = True #@param {type:"boolean"}
right_arm = True #@param {type:"boolean"}
left_arm = True #@param {type:"boolean"}
right_forearm = True #@param {type:"boolean"}
left_forearm = True #@param {type:"boolean"}

segment_angles_var = [right_foot, left_foot, right_shank, left_shank, right_thigh, left_thigh, trunk, right_arm, left_arm, right_forearm, left_forearm]
segment_angles_dict = {'Right foot':right_foot, 'Left foot': left_foot, 'Right shank':right_shank, 'Left shank':left_shank, 'Right thigh':right_thigh, 'Left thigh':left_thigh, 'Trunk':trunk, 'Right arm':right_arm, 'Left arm':left_arm, 'Right forearm':right_forearm, 'Left forearm':left_forearm}
segment_angles_val = [key for key in segment_angles_dict.keys() if segment_angles_dict[key]]

config_dict.get('compute_angles').update({'segment_angles':segment_angles_val})

### Advanced pose settings

In [ ]:
#@markdown Do not run pose estimation again if files already exist:
overwrite_pose = True #@param {type:"boolean"}
config_dict.get('pose_advanced').update({'overwrite_pose':overwrite_pose})

#@markdown <br>

#@markdown Choose whether to save images and video of pose estimation:
save_img = True #@param {type:"boolean"}
save_vid = True #@param {type:"boolean"}
config_dict.get('pose_advanced').update({'save_img':save_img})
config_dict.get('pose_advanced').update({'save_vid':save_vid})

#@markdown <br>

#@markdown Interpolate gaps only if they are smaller than XX frames:
interp_gap_smaller_than = 5 #@param {type:"slider", min:0, max:100, step:1}

#@markdown <br>

#@markdown Choose whether to filter results:
filter = True #@param {type:"boolean"}
filter_type = 'butterworth' #@param ["butterworth", "gaussian", "LOESS", "median"]
config_dict.get('pose_advanced').update({'filter':filter})
config_dict.get('pose_advanced').update({'filter_type':filter_type})

#@markdown <br>

#@markdown - If Butterworth filter is chosen, specify its order and cut-off frequency (in Hz):
order = 4 #@param {type:"integer"}
config_dict.get('pose_advanced').get('butterworth').update({'order':order})
cut_off_frequency = 4 #@param {type:"integer"}
config_dict.get('pose_advanced').get('butterworth').update({'cut_off_frequency':cut_off_frequency})

#@markdown <br>

#@markdown - If Gaussian filter is chosen, specify its sigma kernel (in pixel):
sigma_kernel = 1 #@param {type:"integer"}
config_dict.get('pose_advanced').get('gaussian').update({'sigma_kernel':sigma_kernel})

#@markdown <br>

#@markdown - If LOESS filter is chosen, specify the number of values used:
nb_values_used = 5 #@param {type:"integer"}
config_dict.get('pose_advanced').get('loess').update({'nb_values_used':nb_values_used})

#@markdown <br>

#@markdown - If Median filter is chosen, specify its kernel size (in frames):
kernel_size = 3 #@param {type:"integer"}
config_dict.get('pose_advanced').get('median').update({'kernel_size':kernel_size})

### Advanced angle settings

In [ ]:
#@markdown Choose whether to save images and video of angle estimation:
show_angles_on_img = True #@param {type:"boolean"}
show_angles_on_vid = True #@param {type:"boolean"}
config_dict.get('compute_angles_advanced').update({'show_angles_on_img':show_angles_on_img})
config_dict.get('compute_angles_advanced').update({'show_angles_on_vid':show_angles_on_img})

#@markdown <br>

#@markdown Same angles whether the participant faces left/right.\n
#@markdown Set it to false if you want time series to be continuous even when the participent switches their stance.
flip_left_right = True #@param {type:"boolean"}
config_dict.get('compute_angles_advanced').update({'flip_left_right':flip_left_right})

#@markdown <br>

#@markdown Choose whether to filter results:
filter = True #@param {type:"boolean"}
filter_type = 'butterworth' #@param ["butterworth", "gaussian", "LOESS", "median"]
config_dict.get('compute_angles_advanced').update({'filter':filter})
config_dict.get('compute_angles_advanced').update({'filter_type':filter_type})

#@markdown <br>

#@markdown - If Butterworth filter is chosen, specify its order and cut-off frequency (in Hz):
order = 4 #@param {type:"integer"}
config_dict.get('compute_angles_advanced').get('butterworth').update({'order':order})
cut_off_frequency = 4 #@param {type:"integer"}
config_dict.get('compute_angles_advanced').get('butterworth').update({'cut_off_frequency':cut_off_frequency})

#@markdown <br>

#@markdown - If Gaussian filter is chosen, specify its sigma kernel (in pixel):
sigma_kernel = 1 #@param {type:"integer"}
config_dict.get('compute_angles_advanced').get('gaussian').update({'sigma_kernel':sigma_kernel})

#@markdown <br>

#@markdown - If LOESS filter is chosen, specify the number of values used:
nb_values_used = 5 #@param {type:"integer"}
config_dict.get('compute_angles_advanced').get('loess').update({'nb_values_used':nb_values_used})

#@markdown <br>

#@markdown - If Median filter is chosen, specify its kernel size (in frames):
kernel_size = 3 #@param {type:"integer"}
config_dict.get('compute_angles_advanced').get('median').update({'kernel_size':kernel_size})

## Run analysis

In [ ]:
import logging, logging.handlers
from Sports2D import Sports2D

# Set up logging
logging.basicConfig(format='%(message)s', level=logging.INFO, force=True,
                    handlers = [logging.handlers.TimedRotatingFileHandler(os.path.join(result_path, 'logs.txt'), when='D', interval=7), logging.StreamHandler()])

# Change to result directory
%cd $result_path

# Save the configuration
with open('Config.toml', "w") as config_file:
    toml.dump(config_dict, config_file)

# Execute Sports2D for video
print("Executing Sports2D for video...")
Sports2D.detect_pose('Config.toml')
Sports2D.compute_angles('Config.toml')

print("Execution completed.")

/content/drive/My Drive/Sports2D/Sports2D_results
Executing Sports2D for video...


NumExpr defaulting to 2 threads.


---------------------------------------------------------------------
Detecting pose for video demo.mp4
---------------------------------------------------------------------

No valid CUDA installation found: using OpenVINO backend with CPU.
Selected device: cpu, backend: openvino


CUDA available: True
ONNXRuntime providers: ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'AzureExecutionProvider', 'CPUExecutionProvider']


Downloading: "https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/onnx_sdk/yolox_tiny_8xb8-300e_humanart-6f3252f9.zip" to /root/.cache/rtmlib/hub/checkpoints/yolox_tiny_8xb8-300e_humanart-6f3252f9.zip
100%|██████████| 18.0M/18.0M [00:02<00:00, 7.55MB/s]


load /root/.cache/rtmlib/hub/checkpoints/yolox_tiny_8xb8-300e_humanart-6f3252f9.onnx with openvino backend


...

load /root/.cache/rtmlib/hub/checkpoints/rtmpose-s_simcc-body7_pt-body7-halpe26_700e-256x192-7f134165_20230605.onnx with openvino backend


Processing demo.mp4: 100%|██████████| 54/54 [00:57<00:00,  1.06s/it]
--> Output video saved to /content/drive/My Drive/Sports2D/Sports2D_results/video_results/demo_pose.mp4.
--> Output images saved to /content/drive/My Drive/Sports2D/Sports2D_results/video_results/demo_img.
Sorting people across frames.
2 persons found.
Person 0: Interpolating missing sequences if they are smaller than 5 frames.
Person 0: Filtering with Butterworth filter, 4th order, 3 Hz..
Person 0: Saving csv position file in /content/drive/My Drive/Sports2D/Sports2D_results/video_results/demo_person0_points.csv.
Person 1: Interpolating missing sequences if they are smaller than 5 frames.
Person 1: Filtering with Butterworth filter, 4th order, 3 Hz..
Person 1: Saving csv position file in /content/drive/My Drive/Sports2D/Sports2D_results/video_results/demo_person1_points.csv.
Pose detection and analysis completed.
Pose detection took 160.51 s.


---------------------------------------------------------------------
Com

Processing frames from 0 to 54
Loaded data for 2 persons


Joint and segment computation took 34.96 s.


Video processing completed
Execution completed.


# Retrieve results

## Show video

In [ ]:
import os
import subprocess
from IPython.display import display, HTML

#@markdown Display last uploaded video after analysis:
show_last = 'Yes' #@param ["Yes", "No"]
#@markdown If "No", please type in the video you wish to display (check left pane for a list of them):
show_video_name = '' #@param {type:"string"}
pose_model = 'RTMPose'

pose_path = os.path.join(result_path, 'video_results')

def show_local_mp4_video(file_name, width=640, height=480):
    import io
    import base64
    video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
    return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

if show_last == 'Yes':
    vid_to_show = os.path.join(pose_path, os.path.splitext(vids[-1])[0] + '_' + pose_model + os.path.splitext(vids[-1])[1])
    if not os.path.isfile(vid_to_show):
        vid_to_show = os.path.join(pose_path, os.path.splitext(vids[-1])[0] + '_' + pose_model + '.mp4')
else:
    vid_to_show = os.path.join(pose_path, show_video_name)

if not os.path.isfile(vid_to_show):
    vid_to_show = os.path.join('/content', os.path.basename(vid_to_show))

print(f"Attempting to show video: {vid_to_show}")

if os.path.isfile(vid_to_show):
    # Check that the codec can be read in Colab
    codec = subprocess.run(['ffprobe', '-v', 'error', '-select_streams', 'v:0',
                            '-show_entries', 'stream=codec_name', '-of',
                            'default=noprint_wrappers=1:nokey=1', vid_to_show],
                           capture_output=True, text=True).stdout.strip()

    if codec == 'mpeg4':
        print('Codec not supported: converting video...')
        vid_converted = os.path.join('/content', os.path.basename(vid_to_show))
        result = subprocess.run(['ffmpeg', '-i', vid_to_show, '-c:v', 'libx264',
                                 vid_converted, '-y', '-loglevel', 'quiet'],
                                capture_output=True, text=True)

        if result.returncode == 0:
            print('Conversion successful.')
            vid_to_show = vid_converted
        else:
            print(f'Conversion failed. Error: {result.stderr}')
            print('Attempting to display original video...')

    try:
        display(show_local_mp4_video(vid_to_show, width=640, height=480))
    except Exception as e:
        print(f"Error displaying video: {str(e)}")
        print(f"Video path: {vid_to_show}")
        print(f"Video exists: {os.path.exists(vid_to_show)}")
        if os.path.exists(vid_to_show):
            print(f"Video size: {os.path.getsize(vid_to_show)} bytes")

Attempting to show video: /content/Sports2D/Sports2D_results/video_results/demo_RTMPose.mp4
Codec not supported: converting video...
Conversion successful.


## Plot angles

In [ ]:
import pandas as pd
import plotly.express as px
import os

#@markdown Display knee angle results of the first person detected in the last uploaded video:
plot_last = 'Yes' #@param ["Yes", "No"]
person_id = 'person0' #@param {type:"string"}
#@markdown </br>

#@markdown If "No", please type in the name of the csv file you wish to plot, either of positions or angles (check left pane for a list of them):
plot_csv_name = 'Your_csv_file_name.csv' #@param {type:"string"}
#@markdown And type in the names of the variables you are interested in (see columns of the csv file):
plot_variable_names = "['Right knee','Left knee']" #@param {type:"string"}

#@markdown </br>

#@markdown Display table too:
display_table = 'Yes' #@param ["Yes", "No"]

if plot_last == 'Yes':
    csv_to_show = os.path.join(pose_path, os.path.splitext(vids[-1])[0] + '_' + person_id + '_angles.csv')
    if not os.path.isfile(csv_to_show):
        csv_to_show = os.path.join(pose_path, os.path.splitext(vids[-1])[0] + '_converted_' + person_id + '_angles.csv')
else:
    csv_to_show = os.path.join(pose_path, plot_csv_name)

if not os.path.isfile(csv_to_show):
    print(f"CSV file not found: {csv_to_show}")
else:
    # Retrieve csv results
    table = pd.read_csv(csv_to_show, index_col=0, header=[0,1,2,3])
    table = table.droplevel([0,1], axis=1)
    try:
        plot_variable_names = eval(plot_variable_names)
        plot_variable_names = ['Time'] + plot_variable_names
        table_select = table[plot_variable_names]
        table_select.columns = [' '.join(col).strip() for col in table_select.columns.values]
        table_select = table_select.set_index(list(table_select)[0])
        fig = px.line(data_frame=table_select, width=1310, height=699, title=os.path.basename(csv_to_show))
        fig.show()
    except Exception as e:
        print(f'Error plotting variables: {str(e)}')

    if display_table == 'Yes':
        display(table)

angs,Time,Right ankle,Left ankle,Right knee,Left knee,Right hip,Left hip,Right shoulder,Left shoulder,Right elbow,...,Left foot,Right shank,Left shank,Right thigh,Left thigh,Trunk,Right arm,Left arm,Right forearm,Left forearm
coords,seconds,dorsiflexion,dorsiflexion,flexion,flexion,flexion,flexion,flexion,flexion,flexion,...,horizontal,horizontal,horizontal,horizontal,horizontal,horizontal,horizontal,horizontal,horizontal,horizontal
0,NaN,-34.762563,9.593451,45.159105,117.804498,50.080806,59.507373,73.078408,-66.047659,97.002518,...,-72.626586,-138.189864,-172.220037,-92.969370,-54.415539,47.445363,-69.971768,-179.970570,27.030750,-156.558661
1,0.033447,-29.192391,6.016898,40.107453,114.147306,42.857215,61.822046,73.246014,-65.835580,103.971888,...,-67.650873,-140.245765,-163.667771,-100.092933,-49.520464,48.624738,-69.704134,-177.178090,34.267753,-153.460337
2,0.066608,-22.671282,3.655871,34.947929,110.556856,35.723249,63.902643,72.808097,-65.431431,110.377277,...,-61.810019,-141.727066,-155.465891,-106.848087,-44.909035,49.921984,-69.763240,-174.243108,40.614038,-149.455404
3,0.099816,-15.859095,3.026388,29.910534,106.986281,28.924929,65.638859,71.344219,-64.525707,115.387594,...,-54.820091,-142.420114,-147.846478,-112.849661,-40.860198,51.363131,-70.430372,-171.024763,44.957222,-143.906465
4,0.133064,-11.199778,3.741562,25.685783,103.253960,22.910600,67.087727,68.716382,-62.667095,117.473159,...,-47.141678,-142.651976,-140.883240,-117.704264,-37.629280,52.856226,-71.898482,-167.384103,45.574676,-136.615081
5,0.166335,-11.252741,4.902874,23.220695,99.134267,18.215751,68.417515,65.061494,-59.390932,114.128581,...,-39.635548,-143.300685,-134.538422,-121.070820,-35.404156,54.245086,-74.225078,-163.212603,39.903503,-127.829319
6,0.199615,-16.386415,5.736752,23.338119,94.475261,15.248716,69.771396,60.694177,-54.445211,102.581434,...,-33.006504,-145.662911,-128.743256,-122.751725,-34.267995,55.398917,-77.306264,-158.484602,25.275170,-118.098323
7,0.232896,-23.845119,5.999103,26.408439,89.269313,14.107279,71.119356,55.973753,-47.986197,82.052741,...,-27.468183,-151.148900,-123.467286,-122.765854,-34.197973,56.272145,-80.899380,-153.303527,1.153361,-108.095727
8,0.266172,-29.154029,5.956864,32.214482,83.647636,14.540829,72.178786,51.190021,-40.602927,56.070954,...,-22.809445,-160.505106,-118.766309,-121.351035,-35.118673,56.902070,-84.701842,-147.900386,-28.630888,-98.520562


## View in Google drive

In [ ]:
#@markdown If you chose "Install once", results are already stored on your Google Drive.\
#@markdown If you chose "Install every time", results will first be copied there.

#@markdown `Warning:` If this leads you to a 404 error, run this cell again after a few seconds.

# Buttons to link to Gdrive
import ipywidgets as widgets
from IPython.display import display, Javascript, clear_output, HTML
from google.colab import drive

# Create button
def Gdrive_results_onbuttonclicked(url):
  with output:
    display(Javascript(f'window.open("{url.tooltip}");'))

if installation_type == 'Install every time':
  connect_to_Gdrive()

os.chdir(result_path)
gdrive_result_id = !xattr -p 'user.drive.id' $result_path
gdrive_result_path = 'https://drive.google.com/drive/folders/' + gdrive_result_id[0]
button = widgets.Button(description="View in Google Drive", tooltip=gdrive_result_path, button_style='info')
button.on_click(Gdrive_results_onbuttonclicked)
output = widgets.Output()
display(button, output)

Button(button_style='info', description='View in Google Drive', style=ButtonStyle(), tooltip='https://drive.go…

Output()

## Download results

In [ ]:
#@markdown Download all results. \
#@markdown Alternatively, you can download them individually after finding them in the left pane.

# Compress result folder
!zip -r /content/Sports2D_results.zip /content/Sports2D/Sports2D_results

# Download zip file
from google.colab import files
files.download('/content/Sports2D_results.zip')

  adding: content/Sports2D/Sports2D_results/ (stored 0%)
  adding: content/Sports2D/Sports2D_results/Config.toml (deflated 64%)
  adding: content/Sports2D/Sports2D_results/video_results/ (stored 0%)
  adding: content/Sports2D/Sports2D_results/video_results/demo_json/ (stored 0%)
  adding: content/Sports2D/Sports2D_results/video_results/demo_json/demo_000030.json (deflated 54%)
  adding: content/Sports2D/Sports2D_results/video_results/demo_json/demo_000052.json (deflated 53%)
  adding: content/Sports2D/Sports2D_results/video_results/demo_json/demo_000048.json (deflated 53%)
  adding: content/Sports2D/Sports2D_results/video_results/demo_json/demo_000006.json (deflated 52%)
  adding: content/Sports2D/Sports2D_results/video_results/demo_json/demo_000038.json (deflated 53%)
  adding: content/Sports2D/Sports2D_results/video_results/demo_json/demo_000024.json (deflated 55%)
  adding: content/Sports2D/Sports2D_results/video_results/demo_json/demo_000016.json (deflated 50%)
  adding: content/Sp

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>